This notebook is based on https://medium.com/analytics-vidhya/building-a-simple-chatbot-in-python-using-nltk-7c8c8215ac6e


https://www.geeksforgeeks.org/removing-stop-words-nltk-python/

y
http://josearcosaneas.github.io/python/r/procesamiento/lenguaje/2017/01/02/procesamiento-lenguaje-natural-0.html

In [1]:
import nltk
import numpy as np
import random
import string #used to process python strings

Importing the corpus to colab

In [2]:
#from google.colab import files
#uploaded = files.upload()

Reading the data


In [3]:
f=open('santander_es.txt','r',errors = 'ignore')

raw=f.read()
raw=raw.lower() # converts to lowercase
nltk.download('punkt') # first-time use only
nltk.download('wordnet') # first-time use only
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TheGlitchCat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\TheGlitchCat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Example of sent_tokens and word_tokens

In [4]:
sent_tokens[:2]

['francisco de paula santander\nir a la navegaciã³n\nir a la bãºsqueda\nfrancisco de paula santander\nsantander by acevedo bernal.jpg\ncoat of arms of new granada.svg\n2.âº presidente de la repãºblica de nueva granada\n10 de marzo de 1832-1 de abril de 1837\npredecesor\tjosã© marã\xada obando\nsucesor\tjosã© ignacio de mã¡rquez\ncoat of arms of gran colombia (1821).svg\npresidente de la gran colombia\n(interino)\n7 de octubre de 1819-20 de febrero de 1827\npredecesor\tsimã³n bolã\xadvar\nsucesor\tsimã³n bolã\xadvar\nescudo ejercito nacional de colombia.svg\ngeneral del ejã©rcito nacional de colombia\n1817-1819\npredecesor\tsimã³n bolivar\nsucesor\tsimã³n bolã\xadvar\ninformaciã³n personal\nnacimiento\t2 de abril de 1792\nbandera de espaã±a villa del rosario, virreinato de nueva granada, imperio espaã±ol\nfallecimiento\t6 de mayo de 1840 (48 aã±os)\nbandera de colombia bogotã¡, repãºblica de nueva granada\ncausa de la muerte\tcoledocolitiasis ver y modificar los datos en wikidata\nlugar

In [5]:
word_tokens[:2]


['francisco', 'de']

Pre-processing the raw text

In [6]:
import nltk

#nltk.download()
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
import nltk
#from nltk.stem.porter import PorterStemmer
#from nltk.stem.Snowball import SnowballStemmer

stopw = [w.encode('utf-8') for w in stopwords.words('spanish')]
punc_lit = [u'.', u'[', ']', u',', u';', u'', u')', u'),', u' ', u'(']
stopw.extend(punc_lit)

In [7]:
from nltk.stem import SnowballStemmer

#print (SnowballStemmer.languages)
 
stemmer_spanish = SnowballStemmer('spanish')


def LemTokens(tokens):
    return [stemmer_spanish.stem(token) for token in tokens if token not in stopw]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

#Keyword matching

Next, we shall define a function for a greeting by the bot i.e if a user’s input is a greeting, the bot shall return a greeting response.ELIZA uses a simple keyword matching for greetings. We will utilize the same concept here.

In [8]:
GREETING_INPUTS = ("hola", "que más!", "cordial saludo", "hey!")

GREETING_RESPONSES = ["hola", "hey", "hola que hace!", "buenas las tenga", "a la orden?", "Es un gusto, pregunte pues!"]

def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)    
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize) #, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]    
    if(req_tfidf==0):
        robo_response=robo_response+"Que pena no le entiendo nada... por favor pregunte otra cosa"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [11]:
flag=True
print("ROBO: Soy un chatbot. Espero resolver todas sus preguntas, para salir escriba chao")
while(flag==True):
    user_response = input("You:")
    user_response=user_response.lower()
    if(user_response!='chao'):
        if(user_response=='gracias' or user_response=='Gracias!' ):
            flag=False
            print("ROBO: De nada..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Nos vemos!")

ROBO: Soy un chatbot. Espero resolver todas sus preguntas, para salir escriba chao
You:quien fue santander 
ROBO: bolã­var logrã³ huir por una ventana, gracias a manuelita saenz, quien lo protegiã³.
You:donde nacio santander? 
ROBO: despuã©s de acciones como termã³pilas de paya, pantano de vargas o gã¡meza, donde fue herido al frente de las tropas, y el cruce del pã¡ramo de pisba, con su ejã©rcito fue la pieza definitiva en el triunfo de bolã­var en la batalla de boyacã¡ en 1819, donde rodeã³ al enemigo y lo persiguiã³ evitando que pudiera reorganizarse.
You:quien fue francisco 
ROBO: bolã­var logrã³ huir por una ventana, gracias a manuelita saenz, quien lo protegiã³.
You:que hizo 
ROBO: pero entonces, de acuerdo con el nombramiento de pã¡ez, enterrã³ su espada en el suelo, declarã³ que preferã­a morir con ella enterrada antes que consentir que el nombramiento no fuera hecho adecuadamente y se ganã³ cierto respeto que tambiã©n ganarã­a luchando al lado de pã¡ez.
You:hola 
ROBO: hola qu